# Workshop 3: Explore NT scenario results, modify assumptions & benchmarking

:::{note} At the end of this notebook, you will be able to:

- Navigate and analyze NT scenario results
- Interpret current discrepancies between model outputs and the TYNDP 2024 NT scenario
- Compare 2030 and 2040 scenarios using PyPSA-Explorer
- Modify model assumptions and generate new scenario results
- Apply the benchmarking framework to identify areas for model improvement

:::

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install # TODO
```
:::

In [ ]:
# uncomment for running this notebook on Colab
# !pip install # TODO

In [ ]:
import os
from datetime import datetime
import pypsa
import zipfile
from urllib.request import urlretrieve


import matplotlib.pyplot as plt

pypsa.options.params.statistics.round = 3
pypsa.options.params.statistics.drop_zero = True
pypsa.options.params.statistics.nice_names = False
plt.rcParams["figure.figsize"] = [14, 7]

In [ ]:
def unzip_with_timestamps(zip_path, extract_to):
    """Unzip a file while preserving original file timestamps."""
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        for member in zip_ref.infolist():
            # Extract the file
            zip_ref.extract(member, extract_to)

            # Get the extracted file path
            extracted_path = os.path.join(extract_to, member.filename)

            # Get the modification time from the zip file
            date_time = datetime(*member.date_time)
            timestamp = date_time.timestamp()

            # Set both access and modification times
            os.utime(extracted_path, (timestamp, timestamp))

In [ ]:
urls = {
    # TODO
}

os.makedirs("data", exist_ok=True)
os.makedirs("scripts", exist_ok=True)
for name, url in urls.items():
    if os.path.exists(name):
        print(f"File {name} already exists. Skipping download.")
    else:
        print(f"Retrieving {name} from GCP storage.")
        urlretrieve(url, name)
        print(f"File available in {name}.")

to_dir = "data/open-tyndp"
if not os.path.exists(to_dir):
    print(f"Unzipping data/open-tyndp.zip.")
    unzip_with_timestamps("data/open-tyndp.zip", "data/open-tyndp")
print(f"Open-TYNDP available in '{to_dir}'.")

print("Done")

# Explore NT scenario results

# Modify assumptions

# Benchmark results

# Solutions

## Task 1: # TODO